In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt

In [2]:
def plotCorrelation(data):
    corr_data = data.copy()
    names = list(corr_data.columns)
    correlations = corr_data.corr().abs()
    fig = plt.figure(figsize=(50, 50))
    ax = fig.add_subplot(111)
    cax = ax.matshow(correlations, vmin=-1, vmax=1)
    # fig.colorbar(cax)
    ticks = np.arange(0,len(names),1)
    ax.set_xticks(ticks)
    ax.set_yticks(ticks)
    ax.set_xticklabels(names)
    ax.set_yticklabels(names)
    plt.show()
    
def reduce(data, threshold=0.9):
    correlations = data.corr().abs()
    upper = correlations.where(np.triu(np.ones(correlations.shape), k=1).astype(np.bool))
    to_drop = [column for column in upper.columns if any(upper[column] > threshold)]
    return data.drop(columns=to_drop)

def inpute(data, feature):
    from sklearn.linear_model import ElasticNet
    from sklearn.neural_network import MLPRegressor
    
    X = data.copy().drop(columns=[feature])
    X = X.select_dtypes(exclude=['object'])
    X = X.fillna(X.median())
    y = data[feature]
    X_train = X[~y.isna()]
    X_test = X[y.isna()]
    y_train = y[~y.isna()]
    y_test = y[y.isna()]
    
    model = MLPRegressor()
    model.fit(X_train, y_train)
    print("Feature: %s | Loss = " % feature, model.loss_)
    filled_gaps = model.predict(X_test)
    for i, index in enumerate(data[feature][data[feature].isna()].index):
        data.at[index, feature] = filled_gaps[i]
    return data

In [3]:
data = pd.read_csv("data/week-one/X_train.csv", parse_dates=True)

In [4]:
data['year'] = pd.DatetimeIndex(pd.to_datetime(data.timestamp)).year
data = data.drop(columns=["timestamp"])
data.head()

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc,year
0,11180,51,30.0,2.0,17.0,1.0,1986.0,2.0,8.0,3.0,...,1,0,7,33,1,9,48,11,9300000,2013
1,23071,48,48.0,9.0,17.0,4.0,2016.0,1.0,10.0,1.0,...,5,0,25,44,0,12,114,13,5277083,2014
2,18942,77,46.0,2.0,16.0,1.0,1978.0,3.0,10.0,NaN,...,0,0,5,22,0,1,33,8,12500000,2014
3,24805,109,54.0,9.0,14.0,1.0,2001.0,3.0,11.0,2.0,...,1,0,1,10,1,0,16,0,13050000,2014
4,14778,33,19.0,2.0,3.0,2.0,1961.0,1.0,6.0,3.0,...,3,0,14,29,1,4,90,6,5500000,2014


In [ ]:
# for column in data.columns:
#     print(column)

In [ ]:
# data.corr()

In [ ]:
# plotCorrelation(data)

In [ ]:
# data.corr()['price_doc'].sort_values(ascending=False).iloc[:20].plot.bar()


In [ ]:
data = reduce(data.drop(columns=['price_doc', 'id']), 0.9)
# data

In [ ]:
data.shape

In [ ]:
# plotCorrelation(data)

In [ ]:
# for i in data.dtypes:
#     print(i)

In [ ]:
for column in data.columns[data.isna().any() == True]:
    data = inpute(data, column)

In [5]:
data.dtypes

id                      int64
full_sq                 int64
life_sq               float64
floor                 float64
max_floor             float64
                       ...   
leisure_count_5000      int64
sport_count_5000        int64
market_count_5000       int64
price_doc               int64
year                    int64
Length: 292, dtype: object

In [6]:
from sklearn.preprocessing import OneHotEncoder

In [7]:
enc = OneHotEncoder()
enc.fit(data.select_dtypes(include=['object']))

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=True)

In [8]:
enc.categories_

[array(['Investment', 'OwnerOccupier'], dtype=object),
 array(['Ajeroport', 'Akademicheskoe', 'Alekseevskoe', "Altuf'evskoe",
        'Arbat', 'Babushkinskoe', 'Basmannoe', 'Begovoe',
        'Beskudnikovskoe', 'Bibirevo', 'Birjulevo Vostochnoe',
        'Birjulevo Zapadnoe', 'Bogorodskoe', 'Brateevo', 'Butyrskoe',
        'Caricyno', 'Cheremushki', "Chertanovo Central'noe",
        'Chertanovo Juzhnoe', 'Chertanovo Severnoe', 'Danilovskoe',
        'Dmitrovskoe', 'Donskoe', 'Dorogomilovo', 'Filevskij Park',
        'Fili Davydkovo', 'Gagarinskoe', "Gol'janovo", 'Golovinskoe',
        'Hamovniki', 'Horoshevo-Mnevniki', 'Horoshevskoe', 'Hovrino',
        'Ivanovskoe', 'Izmajlovo', 'Jakimanka', 'Jaroslavskoe', 'Jasenevo',
        'Juzhnoe Butovo', 'Juzhnoe Medvedkovo', 'Juzhnoe Tushino',
        'Juzhnoportovoe', 'Kapotnja', "Kon'kovo", 'Koptevo',
        'Kosino-Uhtomskoe', 'Kotlovka', "Krasnosel'skoe", 'Krjukovo',
        'Krylatskoe', 'Kuncevo', 'Kurkino', "Kuz'minki", 'Lefortovo',
  

In [9]:
for column in data.select_dtypes(include=['object']).drop(columns=['ecology', 'sub_area', 'product_type']).columns:
    data[column] = data[column].map({'yes':1, 'no':0})

In [10]:
data['ecology'] = data['ecology'].map({'excellent':4,'good':3,'satisfactory':2,'poor':1,'no data':0}) 

In [11]:
data.select_dtypes(include=['object'])

,product_type,sub_area
0,Investment,Chertanovo Severnoe
1,OwnerOccupier,Izmajlovo
2,Investment,Jasenevo
3,Investment,Juzhnoe Butovo
4,Investment,Sviblovo
...,...,...
21324,Investment,Sokol
21325,OwnerOccupier,Poselenie Vnukovskoe
21326,Investment,Otradnoe
21327,OwnerOccupier,Poselenie Sosenskoe


In [12]:
data.describe()

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc,year
count,21329.000000,21329.000000,16833.000000,21213.000000,14610.000000,14610.000000,11764.00000,14610.000000,14610.000000,11829.000000,...,21329.000000,21329.000000,21329.000000,21329.000000,21329.000000,21329.000000,21329.000000,21329.000000,2.132900e+04,21329.000000
mean,15210.359511,54.211590,34.388226,7.670579,12.562423,1.828405,1881.99541,1.904723,6.154346,2.105081,...,10.683529,1.748418,14.947818,30.104740,0.439683,8.581649,52.638145,5.985278,7.092863e+06,2013.451357
std,8780.213841,43.094825,61.100592,5.330294,6.821415,1.480861,443.15836,0.852749,17.795476,0.847742,...,28.237869,5.385443,28.992990,47.167864,0.607400,20.490808,46.177586,4.886269,4.661689e+06,0.961120
min,1.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.00000,0.000000,0.000000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000e+05,2011.000000
25%,7620.000000,38.000000,20.000000,3.000000,9.000000,1.000000,1967.00000,1.000000,1.000000,1.000000,...,1.000000,0.000000,2.000000,9.000000,0.000000,0.000000,11.000000,1.000000,4.740000e+06,2013.000000
50%,15160.000000,49.000000,30.000000,6.000000,12.000000,1.000000,1979.00000,2.000000,6.000000,2.000000,...,2.000000,0.000000,7.000000,16.000000,0.000000,2.000000,47.000000,5.000000,6.265926e+06,2014.000000
75%,22765.000000,63.000000,43.000000,11.000000,17.000000,2.000000,2005.00000,2.000000,9.000000,3.000000,...,5.000000,0.000000,12.000000,28.000000,1.000000,7.000000,75.000000,10.000000,8.300000e+06,2014.000000
max,30473.000000,5326.000000,7478.000000,77.000000,117.000000,6.000000,4965.00000,19.000000,1974.000000,4.000000,...,146.000000,30.000000,151.000000,250.000000,2.000000,106.000000,218.000000,20.000000,9.512250e+07,2015.000000


In [14]:
from scipy import stats

In [24]:
data.fillna(data.median(), inplace=True)
z = pd.DataFrame(dict([(column,abs(stats.zscore(data[column]))) for column in data.select_dtypes(exclude=['object']).columns]))

In [55]:
b = data.loc[~((z > 5).sum(axis=1) > 1)]
b.shape

(18355, 292)

In [54]:
data.loc[b.index]

,id,full_sq,life_sq,floor,max_floor,material,build_year,num_room,kitch_sq,state,...,cafe_count_5000_price_4000,cafe_count_5000_price_high,big_church_count_5000,church_count_5000,mosque_count_5000,leisure_count_5000,sport_count_5000,market_count_5000,price_doc,year
0,11180,51,30.0,2.0,17.0,1.0,1986.0,2.0,8.0,3.0,...,1,0,7,33,1,9,48,11,9300000,2013
1,23071,48,48.0,9.0,17.0,4.0,2016.0,1.0,10.0,1.0,...,5,0,25,44,0,12,114,13,5277083,2014
2,18942,77,46.0,2.0,16.0,1.0,1978.0,3.0,10.0,2.0,...,0,0,5,22,0,1,33,8,12500000,2014
4,14778,33,19.0,2.0,3.0,2.0,1961.0,1.0,6.0,3.0,...,3,0,14,29,1,4,90,6,5500000,2014
5,28741,73,50.0,6.0,17.0,4.0,2015.0,2.0,10.0,1.0,...,1,0,5,15,0,3,35,3,8862309,2015
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21324,7589,45,27.0,5.0,12.0,1.0,1979.0,2.0,6.0,2.0,...,18,6,19,41,0,6,115,13,10800000,2013
21325,17692,78,30.0,17.0,17.0,1.0,2015.0,3.0,10.0,2.0,...,1,0,2,12,0,1,10,0,8122791,2014
21326,23122,48,28.0,9.0,9.0,1.0,1975.0,2.0,6.0,3.0,...,4,0,12,25,1,3,82,5,8900000,2014
21327,5324,44,30.0,6.0,12.0,1.0,1979.0,2.0,6.0,2.0,...,2,0,2,16,1,0,20,4,3790560,2012
